In [1]:
from utils_local import *

1.5.3
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
TensorFlow is not using GPU


In [2]:
import pandas as pd

sub = pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/sample_submission.tsv", sep= "\t", header = None)
sub.columns = ["The Protein ID", "The Gene Ontology term (GO) ID", "Predicted link probability that GO appear in Protein"]
sub.head(5)

,The Protein ID,The Gene Ontology term (GO) ID,Predicted link probability that GO appear in Protein
0,A0A0A0MRZ7,GO:0000001,0.123
1,A0A0A0MRZ7,GO:0000002,0.123
2,A0A0A0MRZ8,GO:0000001,0.123
3,A0A0A0MRZ8,GO:0000002,0.123
4,A0A0A0MRZ9,GO:0000001,0.123


In [3]:

PATHS = {
    "PATH": PATH, 
    "PATH_LABELS": PATH_LABELS,
    "PATH_DATAFRAMES_2D": PATH_DATAFRAMES_2D,
    "CHECKPOINT": {"local":'../results/models_bp_aspect/models_{}_{}', "kaggle": '/kaggle/working/models_{}_{}'},
    "PREDICTIONS": {"local":'../results/preds_bp_aspect/preds_fold{}.tsv', "kaggle": 'preds_fold{}.tsv'},
    "LABELS": {"local":'../results/preds_bp_aspect/labels_used_fold{}.tsv', "kaggle": 'labels_used_fold{}.tsv'},
}

In [4]:
from Bio import SeqIO
print("Loading train set ProtBERT Embeddings...")
fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_train)))
fasta_test= SeqIO.parse(config.test_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_test)))

Loading train set ProtBERT Embeddings...
Total Nb of Elements :  142246
Total Nb of Elements :  141865


In [5]:
weights=pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/IA.txt",sep="\t",header=None,index_col=0).rename(columns={1:"weight"})


In [6]:
 labels = pd.read_csv(config.train_labels_path, sep = "\t")
 labels.shape

(5363863, 3)

In [7]:
 labels= labels.loc[~labels.term.isin(list(weights[weights.weight==0].index))]
 labels.shape

(4861356, 3)

In [8]:
# Directories for the different embedding vectors : 
embeds_map = {
    "T5" : "my-t5embeds",
    "ProtBERT" : "protbert-embeddings-for-cafa5",
    "EMS2" : "cafa-5-ems-2-embeddings-numpy"
}

# Length of the different embedding vectors :
embeds_dim = {
    "T5" : 1024,
    "ProtBERT" : 1024,
    "EMS2" : 1280
}

In [9]:
 LEARNING_RATE=0.001
 WARMUP_STEPS=7
 class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps=1):
        super(CustomSchedule, self).__init__()

        self.initial_lr = tf.cast(initial_lr, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return tf.math.minimum(self.initial_lr, self.initial_lr * (step/self.warmup_steps))  
    
    def get_config(self):
        return {"initial_learning_rate": self.initial_lr}
    

'''
PredictionFnCallback is used for:
1. Loading validation data
2. FOGModel data preparation
3. Prediction
4. Scoring and save

'''

'\nPredictionFnCallback is used for:\n1. Loading validation data\n2. FOGModel data preparation\n3. Prediction\n4. Scoring and save\n\n'

In [10]:
import tensorflow as tf

#models_dp_v0
class MultiLayerPerceptron_v0(tf.keras.Model):

    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
                
         
        self.linear1 = tf.keras.layers.Dense(256)
        #self.activation1 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(128)
        #self.activation2 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(56)
        
        self.linear3 = tf.keras.layers.Dense(num_classes,activation="sigmoid")

    def call(self, x):

        x = self.linear1(x)
        #x = self.activation1(x)
        x = self.linear2(x)
        # x = self.activation2(x)
        x = self.linear3(x)
        return x
import tensorflow as tf



#models_dp_v1
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units, activation='relu')

    def call(self, x):
        residual = x
        x = self.fc1(x)
        x = self.fc2(x)
        x = x + residual
        return x

class MultiLayerPerceptron(tf.keras.Model):
    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(56, activation='relu')
        self.res_block1 = ResidualBlock(56)
        self.res_block2 = ResidualBlock(56)
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.fc3(x)
        return x


#models_dp_v2
class ResidualBlock_v2(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units, activation='relu')

    def call(self, x):
        residual = x
        x = self.fc1(x)
        x = self.fc2(x)
        x = x + residual
        return x

class MultiLayerPerceptron_v2(tf.keras.Model):
    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.res_block1 = ResidualBlock(1024)
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.res_block2 = ResidualBlock(512)
        self.fc2 = tf.keras.layers.Dense(256, activation='relu')
        self.res_block3 = ResidualBlock(256)
        self.fc3 = tf.keras.layers.Dense(128, activation='relu')
        self.fc4 = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, x):
        # x = self.res_block1(x)
        x = self.fc1(x)
        # x = self.res_block2(x)
        # x = self.fc2(x)
        # x = self.res_block3(x)
        # x = self.fc3(x)
        x = self.fc4(x)
        return x




#models_cnn_v0
class CNN1D(tf.keras.Model):  #v0
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        # (batch_size, channels, embed_size)
        self.conv1 = tf.keras.layers.Conv1D(filters=7, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 3, embed_size)
        self.pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        # (batch_size, 3, embed_size/2 = 512)
        self.conv2 = tf.keras.layers.Conv1D(filters=15, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 8, embed_size/2 = 512)
        self.pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        
        self.flatten=tf.keras.layers.Flatten()
        # (batch_size, 8, embed_size/4 = 256)
        self.fc1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=num_classes,activation="sigmoid")

    def call(self, x):
        x = tf.reshape(x, [-1, tf.shape(x)[1],1])
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.pool2(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


#models_cnn_v0
class CNN1D(tf.keras.Model):  #v0
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        # (batch_size, channels, embed_size)
        self.conv1 = tf.keras.layers.Conv1D(filters=7, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 3, embed_size)
        self.pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        # (batch_size, 3, embed_size/2 = 512)
        self.conv2 = tf.keras.layers.Conv1D(filters=15, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 8, embed_size/2 = 512)
        self.pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        
        self.flatten=tf.keras.layers.Flatten()
        # (batch_size, 8, embed_size/4 = 256)
        self.fc1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=num_classes,activation="sigmoid")

    def call(self, x):
        x = tf.reshape(x, [-1, tf.shape(x)[1],1])
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.pool2(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x





# 7. Train the Model

In [11]:
import pandas as pd

#X_train = np.load(os.path.join(PATH_DATAFRAMES,f"embeds_train_{embeddings_source}.npy"))
#X_validate = np.load(os.path.join(PATH_DATAFRAMES,f"embeds_test_{embeddings_source}.npy"))
#y_train = pd.read_pickle(os.path.join(PATH_LABELS,f"labels_train_{   config. num_labels}.pkl"))
#y_validate = pd.read_pickle(os.path.join(PATH_LABELS,f"labels_test_{   config. num_labels}.pkl"))
X=np.load(os.path.join(PATH_DATAFRAMES,f"embeds_{embeddings_source}.npy")).astype(np. float32)
y=pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   config. num_labels}.pkl")).astype(np. float32)

num_classes=y.shape[1]
print(f" Num classes {num_classes}")


 Num classes 1500


In [12]:
X.shape,y.shape

((138924, 1280), (138924, 1500))

In [13]:
w_use=weights.loc[y.columns].values
w_use.shape

(1500, 1)

In [14]:
import tensorflow as tf

from sklearn.model_selection import KFold
n_splits = 5

# Crear una instancia de KFold
kfold = KFold(n_splits=n_splits)



In [15]:
from cafa_evaluation_utils.evaluation import *
tau_arr = np.arange(0.1,0.5,0.05)

class PredictionFnCallback(tf.keras.callbacks.Callback):
    
    def __init__(self,X_validate,y_validate, model=None, verbose=0):
        
               if not model is None: self.model = model
               self.verbose = verbose
               self.X_validate = X_validate
               self.y_validate = y_validate
               self.num_classes=num_classes
            
    def on_epoch_end(self, epoch, logs=None):
       
        y_pred=(self.model.predict(self.X_validate))
        #custom_f1_score(y_validate,y_pred)
        ms = []
        w_use=weights.loc[self.y_validate.columns].values
        ms_   =add_metrics(compute_metrics(y_pred, self.y_validate.values, list(range(self.y_validate.shape[1])), tau_arr , ic_arr=w_use.reshape(-1), n_cpu=0), tau_arr, self.y_validate.values )
        ms.append(ms_.wf.max())
        print(np.mean(ms))
            

def custom_f1_score(y_validate,y_pred):
        #custom_f1_score(y_validate,y_pred)
        ms = []
        w_use=weights.loc[y_validate.columns].values

        ms_   =add_metrics(compute_metrics(y_pred, y_validate.values, list(range(y_validate.shape[1])), tau_arr , ic_arr=w_use.reshape(-1), n_cpu=0), tau_arr, y_validate.values )

        ms.append(ms_.wf.max())
        print(np.mean(ms))


In [16]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
models = []
EPOCH_LOAD = "12"
VALIDATE =True
if TRAIN:
    
    NUM_LABELS_dict= {        "BPO": 2000,
                              "CCO":2000,
                              "MFO": 1000}
    ys={
        "CCO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['CCO']}_CCO_importance.pkl")),
        "MFO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['MFO']}_MFO_importance.pkl")),
        "BPO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['BPO']}_BPO.pkl")),
    }
    for fold,(train_indice, test_indice) in enumerate(kfold.split(range(len(X)))):
        for aspect in [  "BPO", "MFO","CCO",]:
        

            y= ys[aspect]

            #X=np.concatenate([X_train,X_validate])
            #y=pd.concat([y_train,y_validate])

            num_classes=y.shape[1]
            print(f" Num classes {num_classes}")

            # model = CNN1D( num_classes)
            model =MultiLayerPerceptron(  num_classes)
            # Define the optimizer
            #optimizer=tf.keras.optimizers.Adam(learning_rate=CustomSchedule(LEARNING_RATE, WARMUP_STEPS), beta_1=0.9, beta_2=0.98, epsilon=1e-9)
            optimizer = tf.keras.optimizers.Adam()

            # Define the loss function
            loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
            model.compile(optimizer=optimizer,loss=loss_fn,metrics=['binary_accuracy', tf.keras.metrics.AUC()])


            
            checkpoint_path = PATHS["CHECKPOINT"][config.env].format(aspect, fold) + '/modelo-{epoch:02d}'
            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_weights_only=False,
                save_freq='epoch',

            )



            model.fit(x=X[train_indice],y=y.iloc[train_indice],epochs=15,batch_size=32, callbacks=[PredictionFnCallback(X[test_indice],y.iloc[test_indice]),checkpoint_callback],)
            #models.append(model)
            del model


        if VALIDATE:
            print("Validating ...")
            predictions_t=None
  
            for aspect in [  "BPO", "MFO","CCO",]:
                y= ys[aspect]

                loaded_model = tf.keras.models.load_model( PATHS["CHECKPOINT"][config.env].format(aspect,fold)+f'/modelo-{EPOCH_LOAD}')
                
                # Use the loaded model for predictions or further operations
                predictions = loaded_model.predict(X[test_indice])

                custom_f1_score(y.iloc[test_indice],np.round(predictions,3) )

                print(np.round(predictions,3)[:10])
                predictions=np.round(predictions,3)
                predictions=pd.DataFrame(predictions,index=y.iloc[test_indice].index,columns=y.iloc[test_indice].columns,)
                predictions=predictions.stack()
                predictions=predictions[predictions>0]
                print(predictions.shape)
                predictions_t=pd.concat([predictions_t,predictions])
            predictions_t=predictions_t.sort_values(ascending=False).groupby(level=0).head(1500)

            predictions_t.to_csv(PATHS["PREDICTIONS"][config.env].format(fold),header=False, index=True, sep="\t")
            labels.loc[labels.EntryID.isin(y.iloc[test_indice].index)].to_csv(PATHS["LABELS"][config.env].format(fold),sep="\t",index=False)

 Num classes 2000
Epoch 1/15
869/869 [==============================] - 1s 1ms/step loss: 
0.23549931006569078


INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_BPO_0\modelo-01\assets


INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_BPO_0\modelo-01\assets


3474/3474 [==============================] - 33s 9ms/step - loss: 0.0437 - binary_accuracy: 0.9894 - auc: 0.8629
Epoch 2/15
869/869 [==============================] - 1s 1ms/step loss:
0.25304725588900706


INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_BPO_0\modelo-02\assets


...

INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_CCO_4\modelo-14\assets


3474/3474 [==============================] - 75s 21ms/step - loss: 0.0089 - binary_accuracy: 0.9969 - auc_14: 0.9783
Epoch 15/15
869/869 [==============================] - 4s 4ms/step
0.32016990653189126


INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_CCO_4\modelo-15\assets


INFO:tensorflow:Assets written to: ../results/models_bp_aspect/models_CCO_4\modelo-15\assets


3474/3474 [==============================] - 88s 25ms/step - loss: 0.0088 - binary_accuracy: 0.9969 - auc_14: 0.9786
Validating ...
869/869 [==============================] - 4s 4ms/step
0.184166562977525
[[0.627 0.633 0.584 ... 0.001 0.    0.001]
 [0.439 0.408 0.406 ... 0.001 0.003 0.002]
 [0.408 0.504 0.485 ... 0.    0.    0.   ]
 ...
 [0.461 0.388 0.354 ... 0.    0.006 0.002]
 [0.437 0.329 0.327 ... 0.    0.001 0.   ]
 [0.262 0.455 0.444 ... 0.    0.001 0.003]]
(36400872,)
869/869 [==============================] - 3s 3ms/step
0.2224775166902633
[[0.464 0.356 0.496 ... 0.    0.    0.   ]
 [0.461 0.457 0.035 ... 0.    0.    0.   ]
 [0.61  0.485 0.05  ... 0.    0.    0.   ]
 ...
 [0.519 0.509 0.009 ... 0.    0.    0.   ]
 [0.333 0.334 0.041 ... 0.    0.    0.   ]
 [0.687 0.691 0.003 ... 0.    0.    0.   ]]
(7198407,)
869/869 [==============================] - 3s 3ms/step
0.29615371152414677
[[0.509 0.397 0.337 ... 0.    0.    0.   ]
 [0.668 0.554 0.479 ... 0.    0.    0.   ]
 [0.69  0

In [17]:
def get_dataset( datatype, embeddings_source):
        if embeddings_source in ["ProtBERT", "EMS2"]:
            embeds = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_embeddings.npy")
            ids = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
        
        if embeddings_source == "T5":
            embeds = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_embeds.npy")
            ids = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            

        if datatype=="train":
            df_labels=pd.read_pickle(PATH_LABELS)
            
            return embeds,ids,df_labels.loc[ids]
        return embeds,ids
        
 

In [18]:
EPOCH_LOAD = "02"



X_test, ids_test= get_dataset( "test", embeddings_source)

predictions= None
for aspect in [ "CCO", "MFO","BPO",]:
    
    y= ys[aspect]
    predictions_test_df_t =None
    for fold in range(n_splits):

        loaded_model = tf.keras.models.load_model(f'models_{aspect}_fold{fold}/modelo-{EPOCH_LOAD}')
        # Use the loaded model for predictions or further operations

        predictions_test = np.round(loaded_model.predict(X_test),3)
        print(predictions_test.shape)
        print(predictions_test[:10])


        print(len(ids_test),len(y.columns))
        predictions_test_df=pd.DataFrame(predictions_test,index=ids_test,columns=y.columns,)
        print((predictions_test_df==0).sum().max())
        
        
        

        predictions_test_df=predictions_test_df.stack()
        predictions_test_df=predictions_test_df[predictions_test_df>0]

        if predictions_test_df_t is None:
            predictions_test_df_t = predictions_test_df
        else:
            predictions_test_df_t = predictions_test_df_t+ predictions_test_df
                
    predictions_test_df_t=predictions_test_df_t/n_splits
    predictions_test_df=predictions_test_df_t
    
    predictions=pd.concat([predictions,predictions_test_df])
                
     

predictions = predictions.sort_values(ascending=False).groupby(level=0).head(1500)
        
        

            

print(predictions_test_df.shape)


predictions_test_df=predictions [predictions >0]
print(predictions_test_df.shape)

predictions_test_df.to_csv("submission.tsv",header=False, index=True, sep="\t")


df_submission = pd.read_csv("submission.tsv",header=None, sep="\t")
display(df_submission)
                            

OSError: No file or directory found at models_CCO_fold0/modelo-02